In [1]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import os
import pandas as pd
import glob
import math
import time
import sys
from scipy.spatial import cKDTree


In [2]:
def Make_dire(file_path):
    if not os.path.isdir(file_path):
        os.makedirs(file_path)

Dans NN, var en x pour simul y<br>
x : T, S, x, y, z<br>
y : MeltRate<br>
1ere Étape : Preparer les données

In [3]:
def get_depth_temp_sali(df, Data):
    ys, IceDraft = df.ny, df.iceDraft
    zs = np.array(Data.nz)
    N_T, N_S = [], []
    ylast = None
    dataset = Data[['temperatureYZ', 'salinityYZ']].to_dataframe().dropna()
    Index_Y = dataset.index.get_level_values('ny')
    for ind, y in enumerate(ys):
        indz = np.abs(np.array(zs + IceDraft[ind])).argmin()
        if y != ylast:
            Slice = dataset.iloc[Index_Y == y].droplevel('ny')
        Z_N = abs(np.array(Slice.index - indz)).argmin() + Slice.index[0]
        New_T = dataset.temperatureYZ.loc[Z_N, y]
        New_S = dataset.salinityYZ.loc[Z_N, y]
        N_T.append(New_T)
        N_S.append(New_S)
    return np.array(N_T), np.array(N_S)

In [4]:
def New_get_depth_temp_sal(df, Data): ## Method 3
    ys, Icedraft = np.array(df.ny), np.array(df.iceDraft)
    Var = ['temperatureYZ', 'salinityYZ']
    y = xr.DataArray(ys)
    iceZ = xr.DataArray(Icedraft)
    Data = Data.assign_coords({'nz':  - Data.nz})
    Filled = Data[Var].interpolate_na(dim="nz", use_coordinate = True, method = 'linear', fill_value="extrapolate")
    Cur = Filled[Var].interp(ny = y, nz = iceZ, method = 'linear')
    T, S = np.array(Cur.to_array())
    return T, S


In [5]:
def Method1_get_T_S(df, Data):
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    T = np.array(Data.temperatureYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
    S = np.array(Data.salinityYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
    return T, S

In [6]:
#def Method2_get_T_S(df, Data):
#    ys = np.array(df.ny)
#    y = xr.DataArray(ys)
#    T = np.array(Data.temperatureYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
#    S = np.array(Data.salinityYZ.sel(ny = y).mean(dim = 'nz', skipna = True))
#    return T, S

In [7]:
def Method2_get_T_S(df, Data):
    """
    Method returns for each point a list of size Z of values averaged on the Y dimensions
    """
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    zs = np.array(df.iceDraft)
    z = xr.DataArray(zs)
    
    Data = Data.assign_coords({'nz':  - Data.nz})
    T = Data.temperatureYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
                                        method = 'linear', fill_value = 'extrapolate').to_numpy()
    S = Data.salinityYZ.mean(dim = 'ny', skipna= True).interpolate_na(dim = 'nz', 
                                        method = 'linear', fill_value = 'extrapolate').to_numpy()
    Names_T = [f"T_{i}" for i in range(Max)]
    Names_S = [f"S_{i}" for i in range(Max)]
    size = len(df)
    T = pd.DataFrame(np.tile(T, (size, 1)), columns = Names_T)
    S = pd.DataFrame(np.tile(S, (size, 1)), columns = Names_S)
    return pd.concat([T, S], axis=1)

In [117]:
def Method4_get_T_S(df, Data, auto_lite, extent = 40):
    """
    Method returns for each point a list of size Z of values at Y
    """
    ys = np.array(df.ny)
    y = xr.DataArray(ys)
    Data = Data.assign_coords({'nz':  - Data.nz})
    T = np.array(Data.temperatureYZ.sel(ny = y).interpolate_na(dim="nz", method = 'nearest', fill_value="extrapolate"))
    S = np.array(Data.salinityYZ.sel(ny = y).interpolate_na(dim="nz", method = 'nearest', fill_value="extrapolate"))
    
    Max = np.shape(T)[0]
    Names_T = [f"T_{i}" for i in range(Max)]
    Names_S = [f"S_{i}" for i in range(Max)]
    
    New_ind = np.linspace(start=0, stop=144, num = extent, endpoint=False).astype(int)
    NEW_T = [f"T_{i}" for i in New_ind]
    NEW_S = [f"S_{i}" for i in New_ind]
    
    DF_T = pd.DataFrame(T, index=Names_T).T
    DF_S = pd.DataFrame(S, index=Names_S).T
    if auto_lite:
        return pd.concat([DF_T[NEW_T], DF_S[NEW_S]], axis=1)
    else:
        return pd.concat([DF_T, DF_S], axis=1)

In [9]:
def Get_Ground_line(df, Arr):
    Arr = Arr.fillna(-9999)
    a = Arr.stack(grid=['ny', 'nx'])
    Grounds = a[a == -9999]
    Oc = a[a != -9999]
    inds_grounds = Grounds.indexes['grid'].to_frame().values.astype(float)
    inds_oc = df[['ny','nx']]
    Spat = cKDTree(inds_grounds)
    Dists, _ = Spat.query(inds_oc)
    return Dists

In [10]:
def Get_Front_line(df, Arr):
    a = Arr.stack(grid=['ny', 'nx'])
    Oc = a[a == 0]
    ice = a[a != -0]
    inds_oc = Oc.indexes['grid'].to_frame().values.astype(float)
    inds_ice = df[['ny','nx']]
    Spat = cKDTree(inds_oc)
    Dists, _ = Spat.query(inds_ice)
    return Dists

In [107]:
def Compute_slope(Data, df):
    ARR = Data
    ARR = xr.where(ARR < 100, 100, ARR).dropna('nx', 'all').interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate')
    x0, x1, y0, y1 = (ARR.shift(nx = -1, fill_value = np.nan).interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(nx = 1, fill_value = np.nan).interpolate_na(dim = "nx", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(ny = -1, fill_value = np.nan).interpolate_na(dim = "ny", use_coordinate = True, method = 'nearest', fill_value='extrapolate'), 
                      ARR.shift(ny = 1, fill_value = np.nan).interpolate_na(dim = "ny", use_coordinate = True, method = 'nearest', fill_value='extrapolate'))
    dx = (x1 - x0)
    dy = (y1 - y0)
    d = (dx**2 + dy**2)**.5
    
    ys, xs = np.array(df.ny), np.array(df.nx)
    y, x = xr.DataArray(ys), xr.DataArray(xs)
    if len(d.coords) == 3 :
        d = d.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
        DX = dx.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
        DY = dy.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny', 'nTime'], axis = 1)
    else : 
        d = d.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
        DX = dx.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
        DY = dy.sel(nx = x, ny = y).to_dataframe().drop(['nx', 'ny'], axis = 1)
    return d, DX, DY

In [108]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
paths = glob.glob(File_path + f'*{Data_type}.nc')
paths

['/bettik/bouissob/ISOMIP+/Ocean1_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']

In [109]:
File_path = '/bettik/bouissob/FAVIER_2019/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
paths = glob.glob(File_path + '*.nc')
paths

['/bettik/bouissob/FAVIER_2019/CPL_EXP13_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP23_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP11_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP21_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP10_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP20_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP22_rst_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/FAVIER_2019/CPL_EXP12_rst_COM_NEMO-CNRS.nc']

In [ ]:
Method = 4
for path in paths[0:1]:
    if path.split('/')[-2] == 'FAVIER_2019':
        auto_lite = True
    else:
        auto_lite = False
        
    dfT = pd.DataFrame()
    Name = path.split('/')[-1].split('_')[0]
    print(path)
    Data = xr.open_dataset(path)
    Data = Data.assign_coords({'nx': Data.x,
                           'ny': Data.y,
                           'nz': Data.z,
                           'nTime': Data.time})
    T = np.array(Data.nTime.as_numpy())
    Horiz_resol = 2 #km/index
    Time = len(T)
    Start = time.perf_counter()
    T = []
    df = []
    for t in range(Time):
        if (t+1)%int(Time/100) == 0:
            print('Starting {} / {}                       Elapsed time = {} s              ETA : {} s                         '.format(t+1, Time, 
                                                    int(time.perf_counter() - Start), int((Time-t+1) * (time.perf_counter() - Start)/(t+1))), end = '\r')
        df = pd.DataFrame()
        df['meltRate'] = (Data['meltRate'].isel(nTime = t).to_dataframe())['meltRate']
        df['thermalDriving'] = Data['thermalDriving'].isel(nTime = t).to_dataframe()['thermalDriving']
        df['halineDriving'] = Data['halineDriving'].isel(nTime = t).to_dataframe()['halineDriving']
        if Data['iceDraft'].ndim == 2:
            df['iceDraft'] = Data['iceDraft'].to_dataframe()['iceDraft']
            df['bathymetry'] = Data['bathymetry'].to_dataframe()['bathymetry']
#            df['bathymetry'] = Data['bathymetry'].to_dataframe()['bathymetry']
        else:
            df['iceDraft'] = Data['iceDraft'].isel(nTime = t).to_dataframe()['iceDraft']
            df['bathymetry'] = Data['bathymetry'].isel(nTime = t).to_dataframe()['bathymetry']
        df['date'] = t
        df.dropna(inplace = True)
        df.reset_index(inplace=True)

        if Data['iceDraft'].ndim == 2:
            if t == 0 :
                Dist_gl = Get_Ground_line(df, Data.iceDraft)
                Dist_fl = Get_Front_line(df, Data.iceDraft)
                Slope_ice, Slope_iceDraft_x, Slope_iceDraft_y = Compute_slope(Data.iceDraft, df)
                Slope_bathy, Slope_bathymetry_x, Slope_bathymetry_y = Compute_slope(Data.bathymetry, df)
            df['Distances_ground_line'] = Dist_gl
            df['Distances'] = Dist_gl
            df['Distances_front_line'] = Dist_fl
            df['Slope_iceDraft'] = Slope_ice
            df['Slope_iceDraft_x'] = Slope_iceDraft_x
            df['Slope_iceDraft_y'] = Slope_iceDraft_y
            
            df['Slope_bathymetry'] = Slope_bathy
            df['Slope_bathymetry_x'] = Slope_bathymetry_x
            df['Slope_bathymetry_y'] = Slope_bathymetry_y
        else :
            Dist_gl = Get_Ground_line(df, Data.iceDraft.isel(nTime = t))
            df['Distances_ground_line'] = Dist_gl
            df['Distances'] = Dist_gl
            df['Distances_front_line'] = Get_Front_line(df, Data.iceDraft.isel(nTime = t))
            df['Slope_iceDraft'], df['Slope_iceDraft_x'], df['Slope_iceDraft_y'] = Compute_slope(Data.iceDraft.isel(nTime = t), df)
            df['Slope_bathymetry'], df['Slope_bathymetry_x'], df['Slope_bathymetry_y'] = Compute_slope(Data.bathymetry.isel(nTime = t), df)
        
        if Method == 2:
            df_new = Method2_get_T_S(df, Data.isel(nTime = t))
            df = pd.concat([df, df_new], axis = 1)
        if Method == 3 or Method == None:
            T, S = New_get_depth_temp_sal(df, Data.isel(nTime = t))
            df['temperatureYZ'] = T
            df['salinityYZ'] = S
        if Method == 4:
            df_new = Method4_get_T_S(df, Data.isel(nTime = t), auto_lite)
            df = pd.concat([df, df_new], axis = 1)
        dfT = pd.concat([dfT, df], ignore_index = True)

    print('Done in {} s'.format(time.perf_counter() - Start))#, end = '\r')
    #dfT.dropna(inplace = True)
    #dfT.reset_index(drop = True, inplace = True)
    dfT.rename(columns={'nx' :'x', 'ny': 'y'}, inplace=True)
    if Method != None:
        if auto_lite:
            f_path = Bet_path + f'Method_Data/FAVIER_2019/Method_{Method}/'
        else:
            f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}/'
        Make_dire(f_path)
        print(sys.getsizeof(dfT) / 10**6)
        if auto_lite:
            name = path.split('/')[-1].replace(f'_{Data_type}.nc', '')
            pd.DataFrame.to_csv(dfT, f_path + f'{Name}_lite.csv', index = False)
        else:
            pd.DataFrame.to_csv(dfT, f_path + f'{Name}.csv', index = False)
#    else:
#        pd.DataFrame.to_csv(dfT, Bet_path + 'Data/New_data_{}_{}.csv'.format(Name, Data_type), index = False)

/bettik/bouissob/FAVIER_2019/CPL_EXP13_rst_COM_NEMO-CNRS.nc


In [133]:
dfT

,ny,nx,meltRate,thermalDriving,halineDriving,iceDraft,bathymetry,date,Distances_ground_line,Distances,...,S_108,S_111,S_115,S_118,S_122,S_126,S_129,S_133,S_136,S_140
0,1000.0,507000.0,2.049093e-08,0.078403,1.067650,84.569778,104.606598,0,2000.000000,2000.000000,...,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843
1,1000.0,509000.0,2.123374e-08,0.082845,1.128011,74.355858,104.694977,0,4000.000000,4000.000000,...,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843
2,1000.0,511000.0,2.106440e-08,0.083315,1.134089,67.040497,104.878571,0,6000.000000,6000.000000,...,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843
3,1000.0,513000.0,2.235711e-08,0.080723,1.098877,61.584545,105.162506,0,8000.000000,8000.000000,...,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843
4,1000.0,515000.0,2.553422e-08,0.082597,1.119576,60.000000,105.551544,0,8000.000000,8000.000000,...,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843,34.146843
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41623,79000.0,631000.0,4.573603e-08,0.013824,0.193353,85.633018,520.000000,12,76419.892698,76419.892698,...,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093
41624,79000.0,633000.0,5.089622e-08,0.014074,0.196831,85.931480,540.000000,12,78409.183136,78409.183136,...,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093
41625,79000.0,635000.0,4.702027e-08,0.010164,0.142338,85.531181,560.000000,12,80399.004969,80399.004969,...,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093
41626,79000.0,637000.0,1.074700e-07,0.020642,0.287984,87.688797,579.944580,12,82389.319696,82389.319696,...,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093,33.761093


In [4]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
def Convert_to_lite(Bet_path, Data_type, Method, Desired_len = 40, Oc = 'Ocean1'):
    f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}'
    path = f"{f_path}/{Oc}.csv"
    extent = 144
    
    df = pd.read_csv(path, nrows = 1)
    COL = df.columns
    New_ind = np.linspace(start=0, stop=extent, num = Desired_len, endpoint=False).astype(int)
    ALL_ind = [i for i in range(extent)]
    Dropping_ind = [i for i in ALL_ind if i not in New_ind]
    Drop_T = [f"T_{i}" for i in Dropping_ind]
    Drop_S = [f"S_{i}" for i in Dropping_ind]
    
    COL = [i for i in COL if ((i not in Drop_S) and (i not in Drop_T))]
    df = pd.read_csv(path, usecols = COL)
    T = [f"T_{i}" for i in New_ind]
    S = [f"S_{i}" for i in New_ind]
    df.rename(dict(zip(T, [ f"T_{i}" for i in np.arange(0, Desired_len).astype(str) ])), axis = 1, inplace = True)
    df.rename(dict(zip(S, [ f"S_{i}" for i in np.arange(0, Desired_len).astype(str) ])), axis = 1, inplace = True)
    pd.DataFrame.to_csv(df, f"{f_path}/{Oc}_lite.csv", index = False)

for Oc in ['Ocean1', 'Ocean2', 'Ocean3', 'Ocean4']:
    print(f"Starting {Oc}")
    Start = time.perf_counter()
    A = Convert_to_lite(Bet_path, Data_type, Method = 4, Oc = Oc)
    print(f"{time.perf_counter() - Start} s elapsed")

Starting Ocean1
57.92728906699631 s elapsed
Starting Ocean2
68.83987011101271 s elapsed
Starting Ocean3
333.50088120199507 s elapsed
Starting Ocean4
323.15158853099274 s elapsed


In [15]:
A.get_chunk()

,y,x,meltRate,thermalDriving,halineDriving,iceDraft,date,Distances,Slope_iceDraft,Slope_bathymetry,...,S_108,S_111,S_115,S_118,S_122,S_126,S_129,S_133,S_136,S_140
0,1000.0,515000.0,1.093901e-09,0.006880,0.096814,76.00000,0,2000.000000,0.000000,13.900721,...,34.372820,34.372820,34.393467,34.412605,34.430058,34.453327,34.470780,34.494050,34.51150,34.53477
1,1000.0,637000.0,2.408978e-09,0.015139,0.212455,100.27083,0,124000.000000,4.845375,40.295700,...,34.372820,34.372820,34.393467,34.412605,34.430058,34.453327,34.470780,34.494050,34.51150,34.53477
2,3000.0,515000.0,1.129151e-09,0.007101,0.099918,76.00000,0,2000.000000,0.000000,16.309322,...,34.372820,34.372820,34.393467,34.412605,34.430058,34.453327,34.470780,34.494050,34.51150,34.53477
3,3000.0,615000.0,1.928918e-09,0.012117,0.170224,90.30713,0,102000.000000,1.290093,28.304380,...,34.372820,34.372820,34.393467,34.412605,34.430058,34.453327,34.470780,34.494050,34.51150,34.53477
4,3000.0,617000.0,1.883951e-09,0.011834,0.166260,90.29829,0,104000.000000,1.288437,26.577932,...,34.372820,34.372820,34.393467,34.412605,34.430058,34.453327,34.470780,34.494050,34.51150,34.53477
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,69000.0,625000.0,5.093313e-10,0.002630,0.037129,101.60897,28,112285.350781,0.006814,105.273690,...,34.359753,34.359753,34.377670,34.399790,34.433174,34.477684,34.511066,34.555576,34.58896,34.63347
99996,69000.0,627000.0,5.729667e-10,0.003050,0.043048,101.60707,28,114280.357017,0.008504,104.427086,...,34.359753,34.359753,34.377670,34.399790,34.433174,34.477684,34.511066,34.555576,34.58896,34.63347
99997,69000.0,629000.0,8.401959e-10,0.004357,0.061446,101.60234,28,116275.534830,0.006065,109.512930,...,34.359753,34.359753,34.377670,34.399790,34.433174,34.477684,34.511066,34.555576,34.58896,34.63347
99998,69000.0,631000.0,1.571362e-09,0.008342,0.117405,101.60101,28,118270.875536,0.001333,111.625946,...,34.359753,34.359753,34.377670,34.399790,34.433174,34.477684,34.511066,34.555576,34.58896,34.63347


In [4]:
File_path = '/bettik/bouissob/ISOMIP+/'
Bet_path = '/bettik/bouissob/'
Data_type = 'COM_NEMO-CNRS'
Method = 4
f_path = Bet_path + f'Method_Data/{Data_type}/Method_{Method}'
files = glob.glob(f"{f_path}/Ocean1_lite.csv")[0]

In [5]:
df = pd.read_csv(files)


In [9]:
T = [f'T_{i}' for i in range(40)]
S = [f'S_{i}' for i in range(40)]

In [18]:
df.drop(T+S, axis = 1)

,y,x,meltRate,thermalDriving,halineDriving,iceDraft,bathymetry,date,Distances_ground_line,Distances,Distances_front_line,Slope_iceDraft,Slope_iceDraft_x,Slope_iceDraft_y,Slope_bathymetry,Slope_bathymetry_x,Slope_bathymetry_y
0,1000.0,515000.0,1.445518e-09,0.009086,0.127754,82.815510,124.00000,0,2000.0,2000.0,2000.000000,0.000000,0.000000,0.000000,17.934357,17.934357,0.000000
1,1000.0,637000.0,2.533100e-09,0.013837,0.194261,91.276440,579.32086,0,124000.0,124000.0,2000.000000,7.109749,0.000000,-7.109749,40.295700,-40.289978,-0.679138
2,3000.0,515000.0,1.421181e-09,0.008926,0.125516,81.651650,124.00000,0,2000.0,2000.0,2000.000000,0.000000,0.000000,0.000000,18.584671,17.441551,6.417343
3,3000.0,615000.0,2.025993e-09,0.012627,0.177358,89.908270,393.43146,0,102000.0,102000.0,2000.000000,8.020058,0.000000,-8.020058,28.304410,-28.295593,-0.706451
4,3000.0,617000.0,1.932426e-09,0.012012,0.168752,90.281364,407.81378,0,104000.0,104000.0,2000.000000,8.546814,0.000000,-8.546814,26.577932,-26.568542,-0.706451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697915,77000.0,635000.0,2.370411e-07,0.080768,1.095842,100.000000,560.00000,239,122000.0,122000.0,2000.000000,16.847622,-6.927826,15.357330,40.003260,-40.000000,0.510925
697916,77000.0,637000.0,3.454754e-07,0.098296,1.324571,106.927826,580.00000,239,124000.0,124000.0,2828.427125,32.680984,0.000000,32.680984,40.005764,-40.000000,0.679138
697917,77000.0,639000.0,2.071392e-07,0.037772,0.521274,100.000000,600.00000,239,126000.0,126000.0,2000.000000,15.018557,6.927826,13.325249,140.000170,-140.000000,0.220947
697918,79000.0,515000.0,4.374625e-08,0.060759,0.831188,82.566990,124.00000,239,2000.0,2000.0,2000.000000,0.000000,0.000000,0.000000,17.934357,17.934357,0.000000


In [10]:
Arr = df.drop(T, axis = 1)
Arr = Arr.drop(S, axis = 1)
M = Arr.mean().to_frame().T
M[T] = df[T].to_numpy().mean()
M[S] = df[S].to_numpy().mean()
Mean = M.mean()

Std = Arr.std().to_frame().T
Std[T] = df[T].to_numpy().std()
Std[S] = df[S].to_numpy().std()
Std2 = Std.mean()

In [105]:
df.rename(dict(zip(T, [ f"T_{i}" for i in np.arange(0, 40).astype(str) ])), axis = 1, inplace = True)
df.rename(dict(zip(S, [ f"S_{i}" for i in np.arange(0, 40).astype(str) ])), axis = 1, inplace = True)

df.columns

Index(['y', 'x', 'meltRate', 'thermalDriving', 'halineDriving', 'iceDraft',
       'date', 'Distances', 'Slope_iceDraft', 'Slope_bathymetry', 'T_0', 'T_1',
       'T_2', 'T_1', 'T_4', 'T_5', 'T_6', 'T_2', 'T_8', 'T_9', 'T_1', 'T_11',
       'T_12', 'T_13', 'T_4', 'T_15', 'T_16', 'T_17', 'T_5', 'T_19', 'T_20',
       'T_6', 'T_22', 'T_23', 'T_24', 'T_2', 'T_26', 'T_27', 'T_8', 'T_29',
       'T_30', 'T_31', 'T_9', 'T_33', 'T_34', 'T_35', 'T_3', 'T_37', 'T_38',
       'T_11', 'S_0', 'S_1', 'S_2', 'S_3', 'S_4', 'S_5', 'S_6', 'S_7', 'S_8',
       'S_9', 'S_10', 'S_11', 'S_12', 'S_13', 'S_14', 'S_15', 'S_16', 'S_17',
       'S_18', 'S_19', 'S_20', 'S_21', 'S_22', 'S_23', 'S_24', 'S_25', 'S_26',
       'S_27', 'S_28', 'S_29', 'S_30', 'S_31', 'S_32', 'S_33', 'S_34', 'S_35',
       'S_36', 'S_37', 'S_38', 'S_39'],
      dtype='object')

In [55]:
dfT

,y,x,meltRate,thermalDriving,halineDriving,iceDraft,date,Distances,Slope_iceDraft,Slope_bathymetry,...,S_134,S_135,S_136,S_137,S_138,S_139,S_140,S_141,S_142,S_143
0,1000.0,515000.0,1.445518e-09,0.009086,0.127754,82.815514,0,2000.0,0.000000,17.934357,...,34.549431,34.555138,34.560844,34.566551,34.572258,34.577965,34.583672,34.589378,34.595085,34.600792
1,1000.0,637000.0,2.533100e-09,0.013837,0.194261,91.276443,0,124000.0,7.109749,40.295700,...,34.549431,34.555138,34.560844,34.566551,34.572258,34.577965,34.583672,34.589378,34.595085,34.600792
2,3000.0,515000.0,1.421181e-09,0.008926,0.125516,81.651649,0,2000.0,0.000000,18.584671,...,33.906647,33.906715,33.906784,33.906853,33.906921,33.906990,33.907059,33.907127,33.907196,33.907265
3,3000.0,615000.0,2.025993e-09,0.012627,0.177357,89.908272,0,102000.0,8.020058,28.304411,...,33.906647,33.906715,33.906784,33.906853,33.906921,33.906990,33.907059,33.907127,33.907196,33.907265
4,3000.0,617000.0,1.932426e-09,0.012012,0.168752,90.281364,0,104000.0,8.546814,26.577932,...,33.906647,33.906715,33.906784,33.906853,33.906921,33.906990,33.907059,33.907127,33.907196,33.907265
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697915,77000.0,635000.0,2.370411e-07,0.080768,1.095842,100.000000,239,122000.0,16.847622,40.003262,...,33.700478,33.700508,33.700539,33.700569,33.700600,33.700630,33.700661,33.700691,33.700722,33.700752
697916,77000.0,637000.0,3.454754e-07,0.098296,1.324571,106.927826,239,124000.0,32.680984,40.005764,...,33.700478,33.700508,33.700539,33.700569,33.700600,33.700630,33.700661,33.700691,33.700722,33.700752
697917,77000.0,639000.0,2.071392e-07,0.037772,0.521274,100.000000,239,126000.0,15.018557,140.000168,...,33.700478,33.700508,33.700539,33.700569,33.700600,33.700630,33.700661,33.700691,33.700722,33.700752
697918,79000.0,515000.0,4.374625e-08,0.060759,0.831188,82.566994,239,2000.0,0.000000,17.934357,...,33.697174,33.697220,33.697266,33.697311,33.697357,33.697403,33.697449,33.697495,33.697540,33.697586


In [56]:
sys.getsizeof(dfT)

843087520

In [12]:
paths[1:]

['/bettik/bouissob/ISOMIP+/Ocean4_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean3_COM_NEMO-CNRS.nc',
 '/bettik/bouissob/ISOMIP+/Ocean2_COM_NEMO-CNRS.nc']